# EGU y OPT con la misma foto base - Comparación

## Optimized Method 

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import torch

# Función para cargar imágenes y convertirlas en puntos
def load_images_to_points(directory):
    image_files = [f for f in os.listdir(directory) if f.endswith('.png') or f.endswith('.jpg')]
    sets_of_points = []

    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path).convert('L')
        image_np = np.array(image) / 255.0  # Normalizar a [0, 1]

        # Binarizar la imagen
        points = np.column_stack(np.where(image_np > 0.5))
        points = points[::2]
        sets_of_points.append(points)

    print("Listo con los puntos")
    return sets_of_points

# Función para calcular la distancia de Hausdorff media (MHD) entre dos conjuntos de puntos
def calculate_mhd_batch(tensor1, tensor2, device):
    A = tensor1.to(device)
    B = tensor2.to(device)
    if A.size(1) != B.size(1):
        raise ValueError("Ambos grupos de puntos tienen diferentes dimensiones.")

    dist_matrix = torch.cdist(A.unsqueeze(0), B.unsqueeze(0)).squeeze(0)
    fhd = torch.mean(torch.min(dist_matrix, dim=1)[0])
    rhd = torch.mean(torch.min(dist_matrix, dim=0)[0])
    mhd = max(fhd.item(), rhd.item())

    return mhd

# Función para computar y guardar un bloque de la matriz de distancias
def compute_and_save_distance_matrix_block(sets_of_points, block_start, block_size, save_dir, device):
    n = len(sets_of_points)
    end = min(block_start + block_size, n)
    block_height = end - block_start

    # Convertir los sets de puntos a tensores
    tensors = [torch.tensor(points, dtype=torch.float32) for points in sets_of_points]

    # Crear submatriz de distancias con NaN
    distance_matrix_block = np.full((block_height, n), np.nan)

    for i in range(block_height):
        tensor1 = tensors[block_start + i]
        distances = []
        for j in range(n):
            if j >= block_start + i:
                tensor2 = tensors[j]
                mhd = calculate_mhd_batch(tensor1, tensor2, device)
                distances.append(mhd)
            else:
                distances.append(np.nan)

        distance_matrix_block[i, :] = distances

        # Imprimir mensaje cada cinco líneas procesadas
        if (i + 1) % 5 == 0 or i + 1 == block_height:
            print(f"Procesadas las líneas {block_start + i - 4} a {block_start + i}")

    # Guardar la submatriz de distancias en el disco
    block_filename = f'OPT_Distance_Matrix_Block_{block_start}_{end}.npy'
    np.save(os.path.join(save_dir, block_filename), distance_matrix_block)

# Función principal adaptada para Jupyter Lab
def main(directory, start_block=0):
    total_start_time = time.time()

    # Cargar imágenes y convertirlas en conjuntos de puntos
    sets_of_points = load_images_to_points(directory)
    n = len(sets_of_points)
    block_size = 2000  # Ajusta el tamaño del bloque según la memoria disponible

    # Crear directorio para guardar los bloques de la matriz de distancias
    save_dir = 'Z:/data/test/blocks'  # Directorio unificado
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Seleccionar dispositivo (GPU o CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando el dispositivo: {device}")

    for block_start in range(start_block, n, block_size):
        block_start_time = time.time()
        compute_and_save_distance_matrix_block(sets_of_points, block_start, block_size, save_dir, device)
        block_end_time = time.time()
        print(f"Procesado el bloque {block_start} a {min(block_start + block_size, n)} en {block_end_time - block_start_time:.2f} segundos")

    total_end_time = time.time()
    elapsed_time = total_end_time - total_start_time

    print(f"Todos los bloques de la matriz de distancias han sido guardados en '{save_dir}'")
    print(f"Tiempo total de ejecución: {elapsed_time:.2f} segundos")

# Ejecutar la función principal
directory = "Z:/data/test"
start_block = 0
main(directory, start_block)


---
## Old Method - EGU 24

In [ ]:
import os
import time
import numpy as np
from PIL import Image
import torch
from torchvision import transforms

# Función para cargar imágenes y convertirlas en tensores
def load_images_to_tensors(directory, device):
    image_files = [f for f in os.listdir(directory) if f.endswith('.png') or f.endswith('.jpg')]
    tensors = []

    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.ToTensor(),
    ])

    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path)
        image_tensor = transform(image).squeeze().to(device)
        tensors.append(image_tensor)

    print("Imágenes cargadas y convertidas a tensores")
    return tensors

# Función para calcular la distancia de Hausdorff modificada (MHD) entre dos tensores
def calculate_mhd(tensor1, tensor2):
    def tensor_to_points(tensor):
        return torch.nonzero(tensor > 0.5, as_tuple=False).float()

    A = tensor_to_points(tensor1)
    B = tensor_to_points(tensor2)
    if A.size(1) != B.size(1):
        raise ValueError("Ambos conjuntos de puntos tienen diferentes dimensiones.")

    dist_matrix = torch.cdist(A, B)
    fhd = torch.mean(torch.min(dist_matrix, dim=1)[0])
    rhd = torch.mean(torch.min(dist_matrix, dim=0)[0])
    mhd = max(fhd.item(), rhd.item())
    return mhd

# Función para computar y guardar un bloque de la matriz de distancias
def compute_and_save_distance_matrix_block(tensors, block_start, block_size, save_dir, device):
    n = len(tensors)
    end = min(block_start + block_size, n)
    block_height = end - block_start

    # Crear submatriz de distancias con NaN
    distance_matrix_block = np.full((block_height, n), np.nan)

    for i in range(block_height):
        tensor1 = tensors[block_start + i]
        distances = []
        for j in range(n):
            if j >= block_start + i:
                tensor2 = tensors[j]
                mhd = calculate_mhd(tensor1, tensor2)
                distances.append(mhd)
            else:
                distances.append(np.nan)

        distance_matrix_block[i, :] = distances

        # Imprimir mensaje cada cinco líneas procesadas
        if (i + 1) % 5 == 0 or i + 1 == block_height:
            print(f"Procesadas las líneas {block_start + i - 4} a {block_start + i}")

    # Guardar la submatriz de distancias en el disco
    block_filename = f'EGU_Distance_Matrix_Block_{block_start}_{end}.npy'
    np.save(os.path.join(save_dir, block_filename), distance_matrix_block)

# Función principal adaptada para procesar por bloques y medir el tiempo
def main(directory, start_block=0):
    total_start_time = time.time()

    # Seleccionar dispositivo (GPU o CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando el dispositivo: {device}")

    # Cargar imágenes y convertirlas en tensores
    tensors = load_images_to_tensors(directory, device)
    n = len(tensors)
    block_size = 2000  # Ajusta el tamaño del bloque según la memoria disponible

    # Crear directorio para guardar los bloques de la matriz de distancias
    save_dir = 'Z:/data/test/blocks'  # Directorio unificado
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for block_start in range(start_block, n, block_size):
        block_start_time = time.time()
        compute_and_save_distance_matrix_block(tensors, block_start, block_size, save_dir, device)
        block_end_time = time.time()
        print(f"Procesado el bloque {block_start} a {min(block_start + block_size, n)} en {block_end_time - block_start_time:.2f} segundos")

    total_end_time = time.time()
    elapsed_time = total_end_time - total_start_time

    print(f"Todos los bloques de la matriz de distancias han sido guardados en '{save_dir}'")
    print(f"Tiempo total de ejecución: {elapsed_time:.2f} segundos")

# Ejecutar la función principal
directory = "Z:/data/test"
start_block = 0
main(directory, start_block)


---
## Numpy

In [ ]:
import os
import time
import numpy as np
from PIL import Image
from scipy.spatial.distance import cdist

# Función para cargar imágenes y convertirlas en conjuntos de puntos
def load_images_to_points(directory):
    image_files = [f for f in os.listdir(directory) if f.endswith('.png') or f.endswith('.jpg')]
    points_sets = []

    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path).convert('L')
        image_np = np.array(image) / 255.0  # Normalizar a [0, 1]

        # Binarizar la imagen
        binary_array = image_np > 0.5
        points = np.column_stack(np.where(binary_array))
        points_sets.append(points)

    print("Imágenes cargadas y convertidas a conjuntos de puntos")
    return points_sets

# Función para calcular la MHD entre dos conjuntos de puntos
def calculate_mhd_numpy(points1, points2):
    D = cdist(points1, points2)
    forward_hd = np.mean(np.min(D, axis=1))
    reverse_hd = np.mean(np.min(D, axis=0))
    mhd = max(forward_hd, reverse_hd)
    return mhd

# Función para computar y guardar un bloque de la matriz de distancias
def compute_and_save_distance_matrix_block(points_sets, block_start, block_size, save_dir):
    n = len(points_sets)
    end = min(block_start + block_size, n)
    block_height = end - block_start

    # Crear submatriz de distancias con NaN
    distance_matrix_block = np.full((block_height, n), np.nan)

    for i in range(block_height):
        points1 = points_sets[block_start + i]
        distances = []
        for j in range(n):
            if j >= block_start + i:
                points2 = points_sets[j]
                mhd = calculate_mhd_numpy(points1, points2)
                distances.append(mhd)
            else:
                distances.append(np.nan)

        distance_matrix_block[i, :] = distances

        # Imprimir mensaje cada cinco líneas procesadas
        if (i + 1) % 5 == 0 or i + 1 == block_height:
            print(f"Procesadas las líneas {block_start + i - 4} a {block_start + i}")

    # Guardar la submatriz de distancias en el disco
    block_filename = f'NumPy_Distance_Matrix_Block_{block_start}_{end}.npy'
    np.save(os.path.join(save_dir, block_filename), distance_matrix_block)

# Función principal adaptada para procesar por bloques y medir el tiempo
def main(directory, start_block=0):
    total_start_time = time.time()

    # Cargar imágenes y convertirlas en conjuntos de puntos
    points_sets = load_images_to_points(directory)
    n = len(points_sets)
    block_size = 2000  # Ajusta el tamaño del bloque según la memoria disponible

    # Crear directorio para guardar los bloques de la matriz de distancias
    save_dir = 'Z:/data/test/blocks'  # Directorio unificado
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for block_start in range(start_block, n, block_size):
        block_start_time = time.time()
        compute_and_save_distance_matrix_block(points_sets, block_start, block_size, save_dir)
        block_end_time = time.time()
        print(f"Procesado el bloque {block_start} a {min(block_start + block_size, n)} en {block_end_time - block_start_time:.2f} segundos")

    total_end_time = time.time()
    elapsed_time = total_end_time - total_start_time

    print(f"Todos los bloques de la matriz de distancias han sido guardados en '{save_dir}'")
    print(f"Tiempo total de ejecución: {elapsed_time:.2f} segundos")

# Ejecutar la función principal
directory = "Z:/data/test"
start_block = 0
main(directory, start_block)
